Step 1: Install and Import Required Libraries

In [ ]:
!pip install --quiet google-cloud-bigquery

In [ ]:
from google.cloud import bigquery

Step 2: Set Project and Dataset Configuration

In [ ]:
PROJECT_ID = "qwiklabs-gcp-01-79d56652b122"
DATASET_ID = "fraud_lab"

client = bigquery.Client(project=PROJECT_ID)


Step 3: Create BigQuery Dataset (Schema)

In [ ]:
dataset_ref = f"{PROJECT_ID}.{DATASET_ID}"

dataset = bigquery.Dataset(dataset_ref)
dataset.location = "US"

dataset = client.create_dataset(dataset, exists_ok=True)

print(f"Dataset {dataset_ref} is ready.")


Dataset qwiklabs-gcp-01-79d56652b122.fraud_lab is ready.


Step 4: Load Raw Fraud Data from GCS into BigQuery

In [ ]:
raw_table_id = f"{PROJECT_ID}.{DATASET_ID}.fraud_data_raw"

job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
)

source_uri = "gs://labs.roitraining.com/data-to-ai-workshop/fraud_data_raw.csv"

load_job = client.load_table_from_uri(
    source_uri,
    raw_table_id,
    job_config=job_config,
)

load_job.result()

print("Raw fraud data loaded into BigQuery table `fraud_data_raw`.")


Raw fraud data loaded into BigQuery table `fraud_data_raw`.


Step 5. Inspect Raw Data

In [23]:
client.query(f"""
select *
from `{raw_table_id}`
limit 20
""").to_dataframe()


,Applicant_ID,Age,Employment_Status,Income,Number_of_Dependents,Amount_Requested,Previous_Assistance_Received,Previous_Assistance_Date,Supporting_Doc_Verified,Application_Frequency_Last_Year,IP_Address,Device_Type,Application_Date,Fraudulent
0,217,65,Unemployed,28984,4,5872,False,NaT,False,1,156.133.45.45,Mobile,2024-08-18,0
1,226,54,Self-Employed,0,1,6631,False,NaT,False,1,245.13.80.245,Tablet,2024-05-11,0
2,240,26,Self-Employed,64477,5,8612,False,NaT,True,1,213.103.170.95,Mobile,2024-08-14,0
3,252,28,Unemployed,28576,4,2951,False,NaT,True,1,234.179.149.207,Desktop,2024-06-12,0
4,266,43,Employed,44930,5,2324,False,NaT,False,1,66.109.96.227,Mobile,2024-08-16,0
5,281,32,Unemployed,25895,3,5411,False,NaT,False,1,202.173.84.210,Desktop,2024-04-30,1
6,300,30,Unemployed,0,4,2245,False,NaT,True,1,178.95.218.82,Tablet,2024-05-15,0
7,303,25,Self-Employed,36223,5,2409,False,NaT,True,1,202.173.163.25,Tablet,2024-03-24,0
8,330,35,Employed,42432,2,793,False,NaT,True,1,114.135.28.171,Tablet,2024-01-14,0
9,353,46,Self-Employed,0,3,8430,False,NaT,True,1,191.195.123.68,Desktop,2024-07-24,0


Step 6. Inspect Schema (Column Names and Data Types)

In [ ]:
schema_df = client.query(f"""
select
  column_name,
  data_type
from `{PROJECT_ID}.{DATASET_ID}.INFORMATION_SCHEMA.COLUMNS`
where table_name = 'fraud_data_raw'
order by ordinal_position
""").to_dataframe()

schema_df


,column_name,data_type
0,Applicant_ID,INT64
1,Age,INT64
2,Employment_Status,STRING
3,Income,INT64
4,Number_of_Dependents,INT64
5,Amount_Requested,INT64
6,Previous_Assistance_Received,BOOL
7,Previous_Assistance_Date,DATE
8,Supporting_Doc_Verified,BOOL
9,Application_Frequency_Last_Year,INT64


Step 7. Inspect Categorical Values for Encoding

In [ ]:
client.query(f"""
select distinct Employment_Status
from `{raw_table_id}`
order by 1
""").to_dataframe()


,Employment_Status
0,Employed
1,Self-Employed
2,Unemployed


In [ ]:
client.query(f"""
select distinct Device_Type
from `{raw_table_id}`
order by 1
""").to_dataframe()


,Device_Type
0,Desktop
1,Mobile
2,Tablet


Step 8. Create Feature-Engineered Training Table

In [ ]:
training_table_id = f"{PROJECT_ID}.{DATASET_ID}.fraud_training_data"

sql = f"""
create or replace table `{training_table_id}` AS
select
  Applicant_ID
  ,Age
  ,Employment_Status
  ,Income
  ,Number_of_Dependents
  ,Amount_Requested
  ,Previous_Assistance_Date
  ,Application_Frequency_Last_Year
  ,IP_Address
  ,Device_Type
  ,Application_Date
  ,Fraudulent

  -- Step 3a: One-hot encode Employment_Status
  ,CASE WHEN Employment_Status = 'Employed' THEN 1 ELSE 0 END AS emp_employed
  ,CASE WHEN Employment_Status = 'Self-Employed' THEN 1 ELSE 0 END AS emp_self_employed
  ,CASE WHEN Employment_Status = 'Unemployed' THEN 1 ELSE 0 END AS emp_unemployed

  -- Step 3a: One-hot encode Device_Type
  ,CASE WHEN Device_Type = 'Mobile' THEN 1 ELSE 0 END AS device_mobile
  ,CASE WHEN Device_Type = 'Desktop' THEN 1 ELSE 0 END AS device_desktop

  -- Step 3b: Age bins and one-hot encoding
  ,CASE WHEN Age BETWEEN 18 AND 24 THEN 1 ELSE 0 END AS age_18_24
  ,CASE WHEN Age BETWEEN 25 AND 34 THEN 1 ELSE 0 END AS age_25_34
  ,CASE WHEN Age BETWEEN 35 AND 44 THEN 1 ELSE 0 END AS age_35_44
  ,CASE WHEN Age BETWEEN 45 AND 54 THEN 1 ELSE 0 END AS age_45_54
  ,CASE WHEN Age >= 55 THEN 1 ELSE 0 END AS age_55_plus

  -- Step 3c: Income-to-Amount-Requested ratio
  ,SAFE_DIVIDE(Income, Amount_Requested) AS income_to_amount_requested

  -- Step 3d: Time since previous assistance
  ,DATE_DIFF(Application_Date,Previous_Assistance_Date,DAY) AS time_since_previous_assistance

  -- Step 3e: True/False fields to 0s and 1s
  ,CAST(Previous_Assistance_Received AS INT64) AS Previous_Assistance_Received
  ,CAST(Supporting_Doc_Verified AS INT64) AS Supporting_Doc_Verified
FROM `{raw_table_id}`;
"""

client.query(sql).result()
print("fraud_training_data created successfully.")


fraud_training_data created successfully.


Step 9. Validate Final Training Table

In [ ]:
client.query(f"""
select *
from `{training_table_id}`
limit 20
""").to_dataframe()


,Applicant_ID,Age,Employment_Status,Income,Number_of_Dependents,Amount_Requested,Previous_Assistance_Date,Application_Frequency_Last_Year,IP_Address,Device_Type,...,device_desktop,age_18_24,age_25_34,age_35_44,age_45_54,age_55_plus,income_to_amount_requested,time_since_previous_assistance,Previous_Assistance_Received,Supporting_Doc_Verified
0,46297,18,Employed,0,1,4041,2023-05-22,1,131.190.246.109,Mobile,...,0,1,0,0,0,0,0.000000,265,1,0
1,13390,18,Employed,31763,3,8746,2023-09-17,1,250.10.113.98,Mobile,...,0,1,0,0,0,0,3.631717,366,1,0
2,16863,18,Employed,0,4,6543,2023-01-07,2,22.146.141.247,Mobile,...,0,1,0,0,0,0,0.000000,437,1,0
3,17089,18,Employed,0,3,8903,2023-12-18,4,178.95.48.128,Tablet,...,0,1,0,0,0,0,0.000000,216,1,1
4,35458,18,Employed,0,0,698,2024-01-02,5,147.25.144.68,Tablet,...,0,1,0,0,0,0,0.000000,119,1,1
5,19396,18,Employed,0,1,4225,2023-09-08,3,239.108.96.171,Tablet,...,0,1,0,0,0,0,0.000000,261,1,0
6,13213,18,Employed,28620,2,7327,2024-06-13,5,253.215.67.54,Tablet,...,0,1,0,0,0,0,3.906101,116,1,0
7,15651,18,Employed,51257,1,5899,2023-10-02,6,151.125.198.186,Tablet,...,0,1,0,0,0,0,8.689100,367,1,0
8,16057,18,Employed,14144,2,3769,2023-10-14,6,80.46.175.4,Desktop,...,1,1,0,0,0,0,3.752720,342,1,0
9,48457,18,Employed,0,5,6924,2023-05-04,9,128.111.212.248,Desktop,...,1,1,0,0,0,0,0.000000,328,1,0


Step 10. Basic Data Quality Checks

In [ ]:
client.query(f"""
select count(1) AS total_rows, countif(income_to_amount_requested is null) AS null_ratios
from `{training_table_id}`
""").to_dataframe()


,total_rows,null_ratios
0,100000,0


In [ ]:
client.query(f"""
select
  Previous_Assistance_Received,
  Supporting_Doc_Verified
from `{training_table_id}`
limit 20
""").to_dataframe()


,Previous_Assistance_Received,Supporting_Doc_Verified
0,1,0
1,1,0
2,1,0
3,1,1
4,1,1
5,1,0
6,1,0
7,1,0
8,1,0
9,1,0
